# Word Embeddings : le modèle Word2Vec

## Imports

In [5]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [6]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [7]:
infile = f"../../data/corpus/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [8]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [9]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [10]:
len(bigram_phrases.vocab.keys())

2367888

Prenons une clé au hasard :

In [11]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

allant_avec


Le dictionnaire indique le score de cette coocurrence :

In [12]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [14]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [15]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [16]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [17]:
print(corpus[:100])

[['3', 'dun', "'", 'le', 'coin', 'des', 'poetes', '.'], ['i_e', 'spir', 'au_hameau', '.'], ['st', '-', 'lfeer', ',', '23_aout', '.'], ['e', '.', 'lambert', '.'], ['-', 'waudrii', 'de', 'va', 'venir', 'du', '--', 'du', 'dimanche_27', 'aout', '189', '?'], ['les', 'gamins', 'en_criant', 'font', 'rouler', 'des', 'cerceaux', '.'], ['lvbas', ',', 'un', 'char', 'de', 'foin', 'rentre', 'dans', 'une', 'grange', '.'], ['do', 'ouvrier', 'suant', "'", 'fend', 'du', 'bois', 'qu', "'", 'il', 'arrange', 'en', 'tas', ';', 'pres', 'd', "'", 'un', 'fumier', 'ou', 'grognent', 'des', 'po', 'arceaux', '.'], ['voici', 'la', 'menagere', 'allant', 'avec', 'deux', 'seaux', 'a', 'la', 'pompe', 'd', "'", 'ou', 'sort', 'l', "'", 'eau', 'par', 'la', 'gueule', 'etrange', 'd', "'", 'un', 'lion', '.'], ['les', 'canards', 'barbotent', 'dans', 'la', 'fange', '.'], ['un', 'cnat', 'passe', 'parmi_les', 'poules', 'a', 'grands', 'sauts', '.'], ['plus_loin', ',', 'le', 'chien', 'aboi6', 'au', 'passant', 'qu', "'", 'il', 've

## Entrainement d'un modèle Word2Vec sur ce corpus

In [18]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 1min, sys: 483 ms, total: 1min 1s
Wall time: 23.2 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [19]:
outfile = f"../../data/corpus/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [20]:
model = Word2Vec.load("../../data/corpus/newspapers.model")

### Imprimer le vecteur d'un terme

In [21]:
model.wv["ministre"]

array([ 2.169128  , -6.5993385 , -2.4424016 ,  3.2607892 , -3.800031  ,
        3.2084959 ,  2.9995608 , -1.009213  , -1.0310135 , -1.7069305 ,
        2.539556  ,  2.2973402 ,  1.8128042 ,  2.364334  ,  1.240359  ,
       -3.6093893 ,  0.17294087,  1.1363162 ,  1.0014226 ,  0.07618248,
       -1.6461427 ,  2.8236804 ,  1.3302144 , -0.31640854, -0.8378373 ,
        3.0017452 , -2.6807666 ,  1.3076642 , -1.1307893 ,  1.5142076 ,
       -2.6605682 , -4.3628554 ], dtype=float32)

### Calculer la similarité entre deux termes

In [22]:
model.wv.similarity("congo", "exportation")

0.3302796

### Chercher les mots les plus proches d'un terme donné

In [35]:
model.wv.most_similar("progres", topn=10)

[('developpement', 0.8645722270011902),
 ('regime', 0.854623556137085),
 ('respect', 0.8536964654922485),
 ('trafic', 0.8205057382583618),
 ('systeme', 0.8156648278236389),
 ('mouvement', 0.8141113519668579),
 ('pays', 0.8094106316566467),
 ('domaine', 0.8065133094787598),
 ('sort', 0.8063024878501892),
 ('territoire', 0.8062463998794556)]

In [36]:
model.wv.most_similar("politique", topn=10)

[('pensee', 0.8452361822128296),
 ('pratique', 0.8444719314575195),
 ('nation', 0.8364242315292358),
 ('politique_coloniale', 0.8321669101715088),
 ('conclusion', 0.8309684991836548),
 ('la_colonisation', 0.819184422492981),
 ('personnelle', 0.8182772994041443),
 ('volonte', 0.8175395727157593),
 ('liberte', 0.8110671639442444),
 ('conscience', 0.8079628348350525)]

In [23]:
model.wv.most_similar("exportation", topn=10)

[('augmentation', 0.9170979261398315),
 ('importation', 0.9039098024368286),
 ('alcool', 0.8914446234703064),
 ('exploitation', 0.8847424983978271),
 ('amortissement', 0.8832395672798157),
 ('achat', 0.8776701092720032),
 ('ivoire', 0.8743161559104919),
 ('acquisition', 0.8733313679695129),
 ('introduction', 0.8720018267631531),
 ('attribution', 0.8704810738563538)]

In [24]:
model.wv.most_similar("exposition", topn=10)

[('inauguration', 0.8554267287254333),
 ('ecole', 0.8425588607788086),
 ('adjudication', 0.8263784646987915),
 ('escadre', 0.8251222968101501),
 ('ouverture', 0.8231439590454102),
 ('assemblee_generale', 0.8117398619651794),
 ('installation', 0.8087273240089417),
 ('entree', 0.8048309683799744),
 ('administration_communale', 0.7999159097671509),
 ('occasion', 0.7827901244163513)]

In [25]:
model.wv.most_similar("art", topn=10)

[('enseignement', 0.7737184762954712),
 ('architecture', 0.7603649497032166),
 ('alcool', 0.7443354725837708),
 ('avancement', 0.7345094680786133),
 ('education', 0.725412905216217),
 ('assimilation', 0.7248212695121765),
 ('elite', 0.7077756524085999),
 ('admiration', 0.6933848261833191),
 ('etudes', 0.6929312348365784),
 ('inscription', 0.6873064637184143)]

In [26]:
model.wv.most_similar("art_decoratif", topn=10)

[('armee_allemande', 0.9364155530929565),
 ('expos', 0.9158846735954285),
 ('antiquite', 0.9082021117210388),
 ('investigation', 0.9052834510803223),
 ('apport', 0.9033463597297668),
 ('tentat', 0.9018136262893677),
 ('urgente_necessite', 0.9013684391975403),
 ('autre_moitie', 0.900053858757019),
 ('afi', 0.897419273853302),
 ('illustration', 0.8958727121353149)]

In [27]:
model.wv.most_similar("colonisation", topn=10)

[('situation_actuelle', 0.8867377638816833),
 ('sauvegarde', 0.8855992555618286),
 ('coalition', 0.8832297325134277),
 ('fraternite', 0.8769776225090027),
 ('solidarite', 0.8765676617622375),
 ('constante', 0.8760496377944946),
 ('doctrine', 0.874079167842865),
 ('conciliation', 0.8727903962135315),
 ('domination', 0.871848464012146),
 ('tolerance', 0.8716562986373901)]

In [28]:
model.wv.most_similar("indigene", topn=10)

[('impot', 0.8386906385421753),
 ('attrait', 0.8025811314582825),
 ('isolement', 0.7972579002380371),
 ('indemnite', 0.7883970737457275),
 ('attribution', 0.7845531702041626),
 ('espace', 0.7777578234672546),
 ('efficacite', 0.7749679684638977),
 ('ensemble', 0.7710190415382385),
 ('eflet', 0.7676894664764404),
 ('afrique_orientale_allemande', 0.7644388675689697)]

In [29]:
model.wv.most_similar("congolais", topn=10)

[('au_congo', 0.7742052674293518),
 ('anglais', 0.7598860263824463),
 ('congolaises', 0.7512133717536926),
 ('du_congo', 0.7508137822151184),
 ('reformes', 0.7478499412536621),
 ('du_domaine', 0.7459226250648499),
 ('des_droits', 0.7457553744316101),
 ('des_negres', 0.7438285946846008),
 ('controle', 0.7416864037513733),
 ('francais', 0.7406160235404968)]

In [30]:
model.wv.most_similar("ivoire", topn=10)

[('alcool', 0.9015253186225891),
 ('embellissement', 0.8883063197135925),
 ('outillage', 0.8827223777770996),
 ('elevage', 0.880047619342804),
 ('affectation', 0.8775478005409241),
 ('exportation', 0.8743160963058472),
 ('algerie', 0.8722872138023376),
 ('attribution', 0.8722137808799744),
 ('extraction', 0.8690187335014343),
 ('exploitation', 0.8678916692733765)]

In [31]:
model.wv.most_similar("palais", topn=10)

[('passage', 0.827998161315918),
 ('quartier', 0.8277766704559326),
 ('pavillon', 0.8223398327827454),
 ('quai', 0.8043650984764099),
 ('theatre', 0.8005605340003967),
 ('local', 0.7989769577980042),
 ('depart', 0.7940343618392944),
 ('cortege', 0.789416491985321),
 ('cafe', 0.7856259346008301),
 ('salon', 0.784581184387207)]

In [32]:
model.wv.most_similar("exotique", topn=10)

[('filer', 0.9436866044998169),
 ('factionnaire', 0.9393418431282043),
 ('vieil_homme', 0.9392215013504028),
 ('111_1_/', 0.9348193407058716),
 ('impressionnant', 0.9311672449111938),
 ('cn_attendant', 0.9309332966804504),
 ('dtf', 0.9275283217430115),
 ('boniment', 0.9271833896636963),
 ('resine', 0.9265388250350952),
 ('chanta', 0.923577070236206)]

In [33]:
model.wv.most_similar("colonie", topn=10)

[('souverainete', 0.9298473596572876),
 ('civilisation', 0.9116161465644836),
 ('prosperite', 0.8980256915092468),
 ('nation', 0.8930848836898804),
 ('puissance', 0.8720464110374451),
 ('protection', 0.8694509267807007),
 ('conquete', 0.8663945198059082),
 ('religion', 0.8609830737113953),
 ('liberte', 0.8579952120780945),
 ('securite', 0.8559237122535706)]